In [ ]:
import os

os.environ["COHERE_API_KEY"] = 'E2efronXbGqXSR0sclyInSjXptFOjdIjsbCeXFwP'

from langchain_cohere import ChatCohere
from langchain_ollama import ChatOllama

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_pymupdf4llm import PyMuPDF4LLMLoader

In [ ]:
from pydantic import BaseModel, Field

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
XL_PATH = r"../dataset/screening/articles_to_screen.csv"
OUT_DIR = r"../dataset/screening/ai_decision.csv"

In [ ]:
# screened abstracts

df = pd.read_csv(XL_PATH)[["key", "title", "abstract"]]

display(df.head())
print(len(df))

In [ ]:
class OutputSchema(BaseModel):
    thoughts: str = Field(description= "thoughts of the model")
    decision: int = Field(description="1 if the abstract was selected, 0 otherwise")
    reason: str = Field(description = "generate a consise one sentence long reason for the decision")


# llm = ChatCohere(model="command-r", temperature=0.0)
llm = ChatOllama(model="deepseek-r1:32b", temperature=0.0, num_ctx=5_000)
struct_llm = llm.with_structured_output(OutputSchema)

In [ ]:
# messages = [
#     ('system',  "You are a helpful AI agent that assists in accurately screening the article using its abstract. " 
#                 "Based solely on the abstract provided, determine whether the article discusses an APPLICATION OF AI METHODS IN CARBON ION THERAPY? "
#                 "Your decision should be '0' for NO or '1' for YES. Then, generate a concise, one-sentence reason for your decision."
#     ),
#     ('human', "abstract:\n\n title: {title}, \n content: {abstract}")
#   ]

messages = [
    ('system',  "You are a helpful AI assistant that accurately screens and SELECTS articles relevant to given TOPIC, for inclusion in a literature review, based solely on their ABSTRACT. " 
                "Your decision should be '1' for YES or '0' otherwise. Then, generate a concise, one-sentence reason for your decision."
    ),
    ('human', "TOPIC: APPLICATIONS OF AI METHODS IN CARBON ION THERAPY\n\nABSTRACT:\n\n title: {title}, \n content: {abstract}")
  ]

prompt_template = ChatPromptTemplate.from_messages(messages)
chain = prompt_template | struct_llm

decision_df = {"key":[], "title":[], "abstract":[], "decision":[], "reason":[], "thoughts":[]}

rows = [row.to_dict() for i,row in df.iterrows()][:5]

for row in tqdm(rows):
  # key, title, selection = row.to_dict()["key"], row.to_dict()["title"], row.to_dict()["selection"]

  inputs = {key:row[key] for key in ["title", "abstract"]} 

  output = chain.invoke(inputs)

  for key,val in {**row, **dict(output)}.items():
      decision_df[key].append(val)
  
decision_df = pd.DataFrame(decision_df)
if not os.path.exists(OUT_DIR):
   os.makedirs(OUT_DIR)
decision_df.to_csv(os.path.join(OUT_DIR, "ai_decision.csv"), index=False)


In [ ]:
# generating summaries

In [ ]:
# this doesn't work very well
# from langchain_community.document_loaders import PyPDFLoader


# file_path = r"1-s2.0-S1120179724002163-main.pdf"
# loader = PyPDFLoader(file_path, mode='single')
# pages = []
# async for page in loader.alazy_load():
#     pages.append(page)

# manuscript = "\n".join([page.page_content for page in pages])

In [ ]:
# import pymupdf4llm

# file_path = r"1-s2.0-S1120179724002163-main.pdf"
# manuscript = pymupdf4llm.to_markdown(file_path)

# ref_index = manuscript.lower().find("references")

# # Keep everything up to and including "References"
# if ref_index != -1:
#     manuscript = manuscript[:ref_index + len("references")]

# # Now md_text contains only up to the References section
# print(manuscript)

In [ ]:

# class OutputSchema(BaseModel):
#     thoughts: str = Field(description= "thoughts of the model")
#     study_objective: str = Field(description="The objective of the article")
#     ai_methods: str = Field(description="The AI method(s) used in the article in the context of carbon ion therapy. Describe each AI method at least in one sentence")
#     application_domain: str = Field(description="the specific carbon ion therapy application domain")
#     dataset_characteristics: str = Field(description="characteristics of the dataset, including the training, validation, test split; also external validation information, if present")
#     key_findings: str = Field(description="key findings mentioned in the article")
#     pros_and_cons: str = Field(description="strengths and limitations of the study")
#     future_direction: str = Field(description="future directions (if stated)")

# llm = ChatCohere(model='command-r', temperature=0.0)
# # llm = ChatOllama(model="deepseek-r1:32b", temperature=0.0)
# struct_llm = llm.with_structured_output(OutputSchema)

# messages = [
#     ('system',  "You are a helpful AI agent that assists in accurately summarizing the article in a standardized format "
#                 "to be used for the literature review titled 'APPLICATIONS OF AI IN CARBON ION THERAPY'. \n"
#                 "Summarize the article under the following standard headers:\n"
#                 "'1. Study objective', '2. AI methods used', '3. carbon therapy application domain', '4. Dataset characteristics', '5. Key findings', "
#                 "'6. Strengths and limitations', '7. Future directions (if stated)' "
#     ),
#     ('human', "article: \n {article}")
#   ]

# prompt_template = ChatPromptTemplate.from_messages(messages)
# chain = prompt_template | struct_llm


In [ ]:

class OutputSchema(BaseModel):
    thoughts: str = Field(description="thoughts before answering the questions")
    aim: str = Field(description="answer to Q1. What is the aim of this study?")
    category: str = Field(description="answer to Q2. Which of the following categories does this article belong to: treatment planning, tumor control prediction, normal tissue complication prediction, or unknown")
    dataset: str = Field(description="answer to Q3. Describe the dataset and provide the training, validation and test strategy involved in AI modelling, including the corresponding sample counts.")
    ai_methodology: str = Field(description="answer to Q4. Define the AI methodology used by the authors for their analysis")
    pros_and_cons: str = Field(description="answer to Q5. What are the strengths and weaknesses of the methodology followed?")
    results: str = Field(description="answer to Q6. Can you summarize the results in terms of the performance metrics and check how appropriate the metrics were to evaluate the AI model?")
    arguments: str = Field(description="answer to Q7. What are the strong and weak arguments that the authors are pointing out in the discussion?")
    conclusion: str = Field(description="answer to Q8. What is their conclusion, and what are their main arguments to support it?")
    critical_summary: str = Field(description="answer to Q9. Can you generate a CRITICAL_SUMMARY combining all the ANSWERS in 300 characters?")

# llm = ChatCohere(model='command-r', temperature=0.0)
llm = ChatOllama(model="deepseek-r1:32b", temperature=0.0, num_ctx=15_000)
struct_llm = llm.with_structured_output(OutputSchema)

In [ ]:
# from transformers import AutoTokenizer

# import json
# schema_str = json.dumps(OutputSchema.model_json_schema())
# tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-32B")
# len(tokenizer.encode(schema_str))

In [ ]:
system_message = ''''
You are a helpful AI reviewer tasked to perform a concise literature review of an ARTICLE for the specified REVIEW_TOPIC.
Carefully read the entire ARTICLE and generate a CRITICAL_SUMMARY by accurately answering each of the QUESTIONS below.

QUESTIONS:
Q1. What is the aim of this study?
Q2. Which of the following categories does this article belong to: treatment planning, tumor control prediction, normal tissue complication prediction, or unknown
Q3. Describe the dataset and provide the training, validation and test strategy involved in AI modelling, including the corresponding sample counts.
Q4. Define the AI methodology used by the authors for their analysis.
Q5. What are the strengths and weaknesses of the methodology followed?
Q6. Can you summarize the results in terms of the performance metrics and check how appropriate the metrics were to evaluate the AI model?
Q7. What are the strong and weak arguments that the authors are pointing out in the discussion?
Q8. What is their conclusion, and what are their main arguments to support it?

Answer each question solely using the information directly available in the ARTICLE. 
The ANSWERS should be detailed, evidence-based and written in an objective and academic tone appropriate for a scientific literature review. 
If specific information required to answer any question is not present in the ARTICLE, state this clearly instead of speculating.

Can you generate a CRITICAL_SUMMARY combining all the ANSWERS in 300 characters?
'''

human_message = "ARTICLE:\n{article}\n\nREVIEW_TOPIC:{topic}"

In [ ]:
messages = [
    ('system', system_message),
    ('human', human_message)
]

prompt_template = ChatPromptTemplate.from_messages(messages)
chain = prompt_template | struct_llm

In [ ]:
outputs = []

ARTICLE_DIR = r"../dataset/articles"

file_paths = [os.path.join(ARTICLE_DIR, file) for file in os.listdir(ARTICLE_DIR) if file.endswith(".pdf")][:2]

for file_path in tqdm(file_paths):
    loader = PyMuPDF4LLMLoader(file_path, mode='single') #we are loading all the pages of the documents in a single page
    docs = loader.load()
    manuscript = docs[0].page_content
    
    
    # Keep everything except contents inside "References"
    ref_index = manuscript.lower().find("references")
    if ref_index != -1:
        manuscript = manuscript[:ref_index + len("references")]

    output = dict(chain.invoke({"article":manuscript, "topic":"APPLICATIONS OF AI METHODS IN CARBON ION THERAPY"}))
    output["file_path"] = file_path


    break

In [ ]:
output

In [ ]:
# from transformers import pipeline, AutoConfig
# from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
# from langchain_core.prompts import PromptTemplate

# model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B" #"deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"

# config = AutoConfig.from_pretrained(model_name)
# pipe = pipeline("text-generation", model=model_name, max_new_tokens=2048, device=1, do_sample=False) #temperature = 0.0 since do_sample is False
# llm = HuggingFacePipeline(pipeline=pipe)


# # print(config)
